# TITLE

## Import modules

In [9]:
import pandas as pd
import numpy as np
import re

## Create per cell categorical matrix outputs from label dataset

In [4]:
raw_output_path = '../exported_annotations/cellAnnotations.csv'
label_data = pd.read_csv(raw_output_path)

In [6]:
simple_label_dict = {'Data':2,'Header':1,'Notes':0,'Other':0,'MetaTitle':0,'Derived':0,'GroupHead':0}
label_data['AnnotationLabel_Simple'] = label_data['AnnotationLabel'].map(simple_label_dict)

In [7]:
file_names = list(label_data['FileName'].unique())

In [10]:
def column_string_to_num(s):
    n = ord(s[-1]) - 64
    if s[:-1]:
        return 26 * (column_string_to_num(s[:-1])) + n
    else:
        return n

def separate_number_chars(s):
    res = re.split('([-+]?\d+\.\d+)|([-+]?\d+)', s.strip())
    res_f = [r.strip() for r in res if r is not None and r.strip() != '']
    return res_f


# document for loop begins here 
output_data = []
for filename in file_names[0:50]:
    single_document = label_data[label_data['FileName'] == filename]
    sheets = single_document['SheetName'].unique()
    for sheet in sheets:
        # sheet for loop begins here 
        single_document_sheet = single_document[single_document['SheetName'] == sheet]
        
        output_list = []
        max_col = 0
        max_row = 0
        for n in range(0,len(single_document_sheet)): #should be optimized to be vectorized not for loop 
            record = single_document_sheet[['AnnotationAddress','AnnotationLabel_Simple']].iloc[n]
            cell_ref = record['AnnotationAddress']
            col_ref_letter = separate_number_chars(cell_ref)[0]
            col_ref = int(column_string_to_num(col_ref_letter)) -1 #for python 0 index
            row_ref = int(separate_number_chars(cell_ref)[1]) -1 #for python 0 index
            label = record['AnnotationLabel_Simple']
            output_list.append((col_ref, row_ref, label))
            if col_ref > max_col:
                max_col = col_ref
            if row_ref > max_row:
                max_row = row_ref
                
        label_matrix = np.zeros((max_row+1,max_col+1))
        for col_ref,row_ref,label in output_list:
            label_matrix[row_ref,col_ref] = label
        output_data.append((filename,sheet,label_matrix))

In [11]:
output_data[0][2].shape

(44, 19)

## Create multi-dimensional matrix inputs from raw spreadsheet inputs

In [15]:
def check_if_empty(y):
    if str(y) == "nan":
        return 1
    else:
        return 0

raw_input_dir = '../dataset/completed/'
input_data = []
for outputs in output_data:
    filename = outputs[0]
    sheet = outputs[1]
    full_file = raw_input_dir + str(filename)
    df = pd.read_excel(full_file,header=None,sheet_name=sheet)
    
    raw_matrix = np.array(df)
    #print(raw_matrix.shape)
    
    type_matrix = np.array([[type(y).__name__ for y in x] for x in raw_matrix])
    #print(type_matrix.shape)
    
    isempty_matrix = np.array([[check_if_empty(y) for y in x] for x in raw_matrix])
    #print(isempty_matrix.shape)
    
    #TODO
    ## import openpyxl get colour and format 
    
    input_matrix = np.dstack((raw_matrix,type_matrix,isempty_matrix))
    input_data.append((filename,sheet,input_matrix))

In [17]:
input_data[0][2].shape

(44, 19, 3)

## Build CNN

In [ ]:
# TODO

# take the inputs 
# Feed them into a simple Neural Network 
# Make the output later the relevant output data (same dimensions as input)
# Figure out how to deal with variable input lengths 
# Get it to predict the output layers 0,1,2
# See it can overfit with perfect overfitting with simple NN architecture
# See the real performmance with train/test set
# Make the architeture more complex look at CNN standard architectures 
# See how good you can get the perofmrance 